In [1]:
import numpy as np
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow as tf

In [2]:
len_vcb = 366549
seed = 123
max_len = 150

In [3]:
with open('/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/utils/tokenizer_100k.json') as f: 
    data = json.load(f) 
    tok = tokenizer_from_json(data)

In [4]:
### definindo metricas customizadas

def recall(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # transformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_recall = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_pred[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_possibles = y_true[K.equal(y_true,K.constant(i))]/i
        possible_positives = K.sum(equals_possibles)

        recall = true_positives / (possible_positives + K.epsilon())
        macro_recall = macro_recall + recall

    macro_recall = macro_recall/(n_tags-1)  

    return macro_recall

def precision(y_true, y_pred):
    y_pred = tf.cast(K.argmax(y_pred), tf.float32) # trannsformando o vetor de (None, 150, n_tags) para (None, 150)
    macro_precision = 0
    y_pred = y_pred[y_true != 0] # retira zero
    y_true = y_true[y_true != 0] # retira zero
    equals = y_true[K.equal(y_true,y_pred)]  # calcula os verdadeiros positivos, ignorando o zero
    
    for i in range(1,n_tags):
        equals_positives = equals[equals == i]  
        true_positives = K.sum(equals_positives/i) # divide por i, para que os valores se tornem 1

        equals_pred_possibles = y_pred[K.equal(y_pred,K.constant(i))]/i
        predicted_positives = K.sum(equals_pred_possibles)

        precision = true_positives / (predicted_positives + K.epsilon())
        macro_precision = macro_precision + precision

    macro_precision = macro_precision/(n_tags-1)

    return macro_precision

def macro_f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

In [5]:
custom_metrics = {"recall": recall,"precision": precision, "macro_f1": macro_f1}
model = tf.keras.models.load_model(
    '/home/rafael/Documents/Faculdade/ICs/IC Direito/codes/pre-processing/3. Fine-Tuning NER rodando no servidor/Resultados em toda base de SP/SP_TOTAL_1EPOCH.h5',
    custom_objects=custom_metrics)

In [6]:
mato_grosso_df = pd.read_json('/home/rafael/Documents/Faculdade/ICs/IC Direito/data/samples/sample_mato_grosso_edital.json', lines=True)    

In [7]:
mato_grosso_sample = mato_grosso_df.sample(1, random_state=seed)['conteudo'].to_list()
mato_grosso_sample

['Cod. Proc.: 992775 Nr: 19774-85.2015.811.0041 AÇÃO: Execução Fiscal->Processo de Execução->PROCESSO CÍVEL E DO TRABALHO PARTE AUTORA: MUNICÍPIO DE CUIABÁ/MT PARTE(S) REQUERIDA(S): IMOBILIÁRIA SANTA ROSA LTDA. ADVOGADO(S) DA PARTE AUTORA: CEZAR FABIANO MARTINS DE CAMPOS ADVOGADO(S) DA PARTE REQUERIDA: Vistos etc... I - Cite-se a parte executada, por carta, para pagar a dívida, com os juros, multa de mora e encargos indicados na Certidão de Dívida Ativa ou garantir a execução, no prazo de 05 (cinco) dias, sob pena de penhora. II - Para as hipóteses de pronto pagamento, fixo os honorários advocatícios em 10% (dez por cento) sobre o valor da execução. III – Efetuada a citação e decorrido o prazo sem o pagamento do débito, intime-se a parte exequente para indicar bens para a penhora, no prazo de 15 (quinze) dias. IV - Na hipótese da citação por carta restar frustrada, dê-se vista à Fazenda Pública para indicar o endereço atualizado do devedor ou requerer o que entender de direito, no praz

In [8]:
mato_grosso_sample_tokenized = tok.texts_to_sequences(mato_grosso_sample)
pd.DataFrame(mato_grosso_sample_tokenized[0]).T

,0,1,2,3,4,5,6,7,8,9,...,249,250,251,252,253,254,255,256,257,258
0,19275,1342,1,117627,1,645,90,12180,60481,978,...,1704,2,107,2917,1778,364,8,2550,47,3250


In [9]:
mato_grosso_sample_tokenized_padded = pad_sequences(maxlen=max_len,
                                    sequences=mato_grosso_sample_tokenized,
                                    padding="post",
                                    truncating="post")
pd.DataFrame(mato_grosso_sample_tokenized_padded)

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,19275,1342,1,117627,1,645,90,12180,60481,978,...,109,250,33,533,504,389,7,137,23,359


In [10]:
predicted_mato_grosso_sample_tokenized_padded = model(mato_grosso_sample_tokenized_padded)

In [11]:
for i,p in enumerate(predicted_mato_grosso_sample_tokenized_padded[0]):
    if (i < len(mato_grosso_sample_tokenized[0])): 
        print(np.argmax(p),tok.index_word[mato_grosso_sample_tokenized[0][i]])

1 Cod
1 Proc
1 UNK
1 Nr
1 UNK
1 85
1 2015
1 811
1 0041
1 AÇÃO
1 Execução
1 Fiscal
1 Processo
1 de
1 Execução
1 PROCESSO
1 CÍVEL
1 E
1 DO
1 TRABALHO
1 PARTE
1 AUTORA
1 MUNICÍPIO
1 DE
1 UNK
1 MT
1 PARTE
1 S
1 REQUERIDA
1 S
1 IMOBILIÁRIA
1 SANTA
1 ROSA
1 LTDA
1 ADVOGADO
1 S
1 DA
1 PARTE
1 AUTORA
1 CEZAR
1 FABIANO
1 MARTINS
1 DE
1 CAMPOS
1 ADVOGADO
1 S
1 DA
1 PARTE
1 REQUERIDA
1 Vistos
1 etc
1 I
1 Cite
1 se
1 a
1 parte
1 executada
1 por
1 carta
1 para
1 pagar
1 a
1 dívida
1 com
1 os
1 juros
1 multa
1 de
1 mora
1 e
1 encargos
1 indicados
1 na
1 Certidão
1 de
1 Dívida
1 Ativa
1 ou
1 garantir
1 a
1 execução
1 no
1 prazo
1 de
1 05
1 cinco
1 dias
1 sob
1 pena
1 de
1 penhora
1 II
1 Para
1 as
1 hipóteses
1 de
1 pronto
1 pagamento
1 fixo
1 os
1 honorários
1 advocatícios
1 em
1 10
1 dez
1 por
1 cento
1 sobre
1 o
1 valor
1 da
1 execução
1 III
1 –
1 Efetuada
1 a
1 citação
1 e
1 decorrido
1 o
1 prazo
1 sem
1 o
1 pagamento
1 do
1 débito
1 intime
1 se
1 a
1 parte
1 exequente
1 para
1 indicar
1 bens
1 pa